# AreaDetector image data

* used BlueSky to acquire an image from ADSimDetector
* now want to access the image

Following these instructions from NSLS2:  http://nsls-ii.github.io/databroker/assets.html

In [1]:
import databroker
import numpy as np
db = databroker.Broker.named("mongodb_config")

---------
## Use Databroker to get the image data

Get the *header* (`h`) of the last data added to the databroker (alternatively, we could reference this data otherwise, see tutorial above for examples)

To acquire an image, see [Ophyd setup](#Ophyd-setup) and [Bluesky setup](#BlueSky-data-acquisition) below.

In [14]:
h = db[-1]

We want the image from the `adsimdet` object that we counted.  That is `adsimdet.image` which is referenced by the name `adsimdet_image`.  The result of `h.data()` is a [generator](https://docs.python.org/3/glossary.html#generator) so we wrap in a list and pick the first one.  (Raises an exception if there is no image.)

To get the image data, we need to know the name that was used.

In [15]:
h.fields()

{'adsimdet_hdf1_file_name',
 'adsimdet_hdf1_file_path',
 'adsimdet_hdf1_full_file_name',
 'adsimdet_image'}

from this list, we want the `adsimdet_image` field.

In [16]:
image = list(h.data("adsimdet_image"))[0]

In [17]:
image

<Frames>
Length: 1 frames
Frame Shape: 1024 x 1024
Pixel Datatype: uint8

It should have one frame, 1k by 1k, 8-bit.  Do some simple math on the image, such as image min/max/mean/std. dev.

In [18]:
np.min(image), np.max(image), np.mean(image), np.std(image)

(0, 248, 124.0, 73.86474125047755)

prepare to visualize the image

In [ ]:
# TBA

---------------

## Ophyd setup

In [2]:
from ophyd import Component, SingleTrigger, AreaDetector, SimDetector
from ophyd import HDF5Plugin, ImagePlugin
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite

In [3]:
image_file_path = "/tmp/simdet/%Y/%m/%d/"

In [4]:
class MyHDF5Plugin(HDF5Plugin, FileStoreHDF5IterativeWrite):
    
    file_number_sync = None
    
    def get_frames_per_point(self):    # AD 2.5
        return self.parent.cam.num_images.get()

In [5]:
class MySingleTriggerHdf5SimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")
    hdf1 = Component(
        MyHDF5Plugin,
        suffix='HDF1:', 
        root='/' ,                              # for databroker
        write_path_template=image_file_path,    # for EPICS AD
    )

In [6]:
_ad_prefix = "otzSIM1:"
adsimdet = MySingleTriggerHdf5SimDetector(_ad_prefix, name='adsimdet')
#
# define these so something appears in the event stream
adsimdet.hdf1.read_attrs.append("file_name")
adsimdet.hdf1.read_attrs.append("file_path")
adsimdet.hdf1.read_attrs.append("full_file_name")
adsimdet.read_attrs.append("hdf1")

In [7]:
adsimdet.image.name

'adsimdet_image'

In [8]:
adsimdet.image

ImagePlugin(prefix='otzSIM1:image1:', name='adsimdet_image', parent='adsimdet', read_attrs=[], configuration_attrs=['configuration_names', 'port_name', 'asyn_pipeline_config', 'blocking_callbacks', 'enable', 'nd_array_port', 'plugin_type'])

## BlueSky data acquisition

In [9]:
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky import SupplementalData

RE = RunEngine({})
RE.md['purpose'] = 'documentation'

import databroker
db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)

sd = SupplementalData()
RE.preprocessors.append(sd)

In [10]:
count_time = 0.2
adsimdet.cam.stage_sigs["acquire_time"] = count_time
adsimdet.describe_configuration()

OrderedDict([('adsimdet_cam_acquire_period',
              {'source': 'PV:otzSIM1:cam1:AcquirePeriod_RBV',
               'dtype': 'number',
               'shape': [],
               'precision': 3,
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0}),
             ('adsimdet_cam_acquire_time',
              {'source': 'PV:otzSIM1:cam1:AcquireTime_RBV',
               'dtype': 'number',
               'shape': [],
               'precision': 3,
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0}),
             ('adsimdet_cam_image_mode',
              {'source': 'PV:otzSIM1:cam1:ImageMode_RBV',
               'dtype': 'integer',
               'shape': [],
               'units': None,
               'lower_ctrl_limit': None,
               'upper_ctrl_limit': None,
               'enum_strs': ['Single', 'Multiple', 'Continuous']}),
             ('adsimdet_cam_manufacturer',

In [11]:
RE(bp.count([adsimdet]))

('04b0a9ae-cd46-419e-a8c8-63ce5c294c50',)

In [12]:
cfg = adsimdet.hdf1.read_configuration()
file_name = cfg["adsimdet_hdf1_full_file_name"]['value']
print(file_name)

/tmp/simdet/2018/07/25/71206f16-5969-48cd-9596_000000.h5


In [13]:
for i, ev in enumerate(db[-1].events()):
    print(i, ev["data"][adsimdet.hdf1.name+"_full_file_name"])

0 /tmp/simdet/2018/07/25/71206f16-5969-48cd-9596_000000.h5
